In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import html2text
import shutil

browser = webdriver.Chrome(executable_path='chromedriver.exe') 
#url = "svn.oli.cmu.edu/svn/content/editor/projects/echo-oli-cmu-edu/foundations-zxfk9cu6/branches/v_1_0-echo/content/x-oli-workbook_page/"
url = "svn.oli.cmu.edu/svn/content/editor/projects/echo-oli-cmu-edu/foundations-zxfk9cu6/branches/v_1_3-echo/content/x-oli-workbook_page/"
text = browser.get("https://bgaind:rGCpX8Jad6_q@" + url)
text = browser.page_source

soup = BeautifulSoup(text, 'html.parser')

content = []
titles = []
cur_urls = []
for i in soup.find("ul").findAll("li"):
    cur_url = i.find("a")["href"]
    cur_urls.append(cur_url)
    text = browser.get("https://bgaind:rGCpX8Jad6_q@" + url + "/" + cur_url)
    text = browser.page_source
    content.append(text)
    soup1 = BeautifulSoup(text, 'html.parser')
    titles.append(soup1.find("title").string.strip().replace(":", "-").replace("?", ""))

In [8]:
DIR = "data1/"
manual_titles = []
f = open("modules", "r")
modules_lines = f.readlines()
f.close()
#shutil.rmtree(DIR)
os.mkdir(DIR)
cur_unit = ""
cur_module = ""
cur_chapter_idx = 0
asd = ""
h = html2text.HTML2Text()
h.ignore_emphasis = True
all_content_parsed = []
for i in modules_lines:
    line = i.strip()
    if line.find("Unit") != -1:
        os.mkdir(DIR + line)
        cur_unit = line
    elif line.find("Module") != -1:
        os.mkdir(DIR + cur_unit + "/" + line)
        cur_module = line
        cur_chapter_idx = 0
    else:
        if line.find("Overview,") !=-1 or line.find("Summary,") !=-1:
            id1 = line.split(",")[1]
            name = line.split(",")[0]
            for c in content:
                soup1 = BeautifulSoup(c, 'html.parser')
                if soup1.find("ul")!= None and soup1.find("ul").has_attr("id") and soup1.find("ul")["id"] == id1:
                    f = open(DIR + "/" + cur_unit + "/" + cur_module + "/" + str(cur_chapter_idx+1) + "-" + name, "wb")
                    c_write = h.handle(c)
                    cidx = c_write.find("This XML file does not appear to have any")
                    c_write = c_write[:cidx].encode("utf-8")
                    f.write(c_write)
                    f.close()
                    all_content_parsed.append(c_write)
                    cur_chapter_idx += 1
                    break
                for p in soup1.findAll("p"):
                    if p["id"] == id1:
                        f = open(DIR + "/" + cur_unit + "/" + cur_module + "/" + str(cur_chapter_idx+1) + "-" + name, "wb")
                        c_write = h.handle(c)
                        cidx = c_write.find("This XML file does not appear to have any")
                        c_write = c_write[:cidx].encode("utf-8")
                        f.write(c_write)
                        f.close()
                        all_content_parsed.append(c_write)
                        cur_chapter_idx += 1
                        break
        elif line not in titles or titles.count(line) > 1:
            print(line)
        else:
            idx = titles.index(line)
            c = content[idx]
            f = open(DIR + "/" + cur_unit + "/" + cur_module + "/" + str(cur_chapter_idx+1) + "-" + line, "wb")
            c_write = h.handle(c)
            cidx = c_write.find("This XML file does not appear to have any")
            c_write = c_write[:cidx].encode("utf-8")
            all_content_parsed.append(c_write)
            f.write(c_write)
            f.close()
            cur_chapter_idx += 1

In [1]:
import re
import PyPDF2
import sys
import fitz
import json
from nltk.tokenize import sent_tokenize, word_tokenize

def parseIndexText(text):
    '''
    Given Index of the format
    `Cloud Computing ................. 74, 81`
    It will extract as ["Concept Page Numbers", ..]
    '''
    contents = text.splitlines()
    content_with_pns = []
    for i in range(len(contents)):
        # check if the next line is numbers only
        if (len(content_with_pns) > 1) and (re.search('[0-9]+',content_with_pns[-1]) is None):
            content_with_pns[-1] =  content_with_pns[-1] + " " + contents[i]
        elif re.search('[a-zA-Z]+', contents[i]) is not None:   
            # if not add to list
            content_with_pns.append(contents[i])
        elif len(content_with_pns) > 1:
            # if yes append to last element for format `Cloud Computing ...... 74, 81 \n 32,22`
            temp = content_with_pns[-1]  
            temp = temp + contents[i]
            content_with_pns[-1] = temp
    return content_with_pns

    
def extractConceptsFromIndexPage(content_with_pns):
    '''
    Extract concepts given the index page of a text book
    '''
    concept_dict = {}

    for content in content_with_pns:
        concepts = content.split(',')
        curr_concepts = []
        curr_pages = []
        for concept in concepts:
            # name of the concepts
            if re.search('[a-zA-Z]+', concept) is not None:
                curr_concepts.append(concept.strip())
            else:
            # page number of concept
                concept = concept.strip()
                if len(concept) > 0:
                    curr_pages.append(concept.replace("–","-").strip())

        if len(curr_pages) > 0:
            concept_dict.update({curr_concept:curr_pages for curr_concept in curr_concepts})

    return concept_dict


# Entry Point
def extractConceptFromIndex(path='', filename="Cloud Computing Bible.pdf", indexPages=(497,527), output=""):
    '''
    To load data from pdf
    Args : filename, name of pdf file
    Rets : the list of concepts from the PDF
    '''
    # Ref : https://www.geeksforgeeks.org/working-with-pdf-files-in-python/
    doc = fitz.open(filename)
    all_concepts_dict = {}

    for pageNumber in range(*indexPages):
        page = doc.loadPage(pageNumber)
        text = page.getText()
        content_with_pns = parseIndexText(text)
        all_concepts_dict.update(extractConceptsFromIndexPage(content_with_pns))
    
    json_dict = json.dumps(all_concepts_dict)
    f = open(output + filename+"_concepts.json", "w+")
    f.write(json_dict)
    f.close()


    # return {concepts:[page numbers, ....]}
    concept_names_raw = list(all_concepts_dict.keys())
    concept_names = []
    for concept_name in concept_names_raw:
        if (concept_name not in concept_names) and (concept_name+"s" 
                    not in concept_names) and (concept_name[:-1] not in concept_names):
            concept_names.append(concept_name)

    return concept_names

In [2]:
concept_names = extractConceptFromIndex(filename="Foundations of Data Science - Cornell CS.pdf", indexPages=(465, 469))

In [3]:
concept_names

['2-universal',
 '4-way independence',
 'Aﬃnity matrix',
 'Algorithm greedy k-clustering',
 'k-means',
 'singular value decomposition',
 'Almost surely',
 'Anchor Term',
 'Aperiodic',
 'Arithmetic mean',
 'Bad pair',
 'Bayes rule',
 'Bayesian',
 'Bayesian network',
 'Belief Network',
 'belief propagation',
 'Bernoulli trials',
 'Best ﬁt',
 'Bigoh',
 'Binomial distribution',
 'approximated by Poisson',
 'boosting',
 'Branching process',
 'Cartesian coordinates',
 'Cauchy-Schwartz inequality',
 'Central Limit Theorem',
 'Characteristic equation',
 'Characteristic function',
 'Chebyshev’s inequality',
 'Chernoﬀ bounds',
 'Clustering',
 'k-center criterion',
 'Sparse Cuts',
 'CNF CNF-sat',
 'Cohesion',
 'Combining expert advice',
 'Commute time',
 'Conditional probability',
 'Conductance',
 'Coordinates Cartesian',
 'polar',
 'Coupon collector problem',
 'Cumulative distribution function',
 'Current probabilistic interpretation',
 'Cycles',
 'emergence',
 'number of',
 'Data streams counti

In [5]:
concept_names = extractConceptFromIndex(filename="Foundations of Data Science - Cornell CS.pdf", indexPages=(465, 469))
concept_names += extractConceptFromIndex(filename="Cloud Computing Bible.pdf", indexPages=(497,527))
concept_names += extractConceptFromIndex(filename="pythondatasciencehandbook.pdf", indexPages=(534, 547))

concepts_found = set()
for c in all_content_parsed:
    ccur = c.decode()
    for concept in concept_names:
        if ccur.find(concept) != -1:
            concepts_found.add(concept)
f = open("concepts", "w")
for i in concepts_found:
    f.write(i + "\n")
f.close()

mupdf: expected object number


In [4]:
concept_names = extractConceptFromIndex(filename="pythondatasciencehandbook.pdf", indexPages=(534, 547))

mupdf: expected object number


In [5]:
concept_names

['Symbols %automagic',
 '%cpaste',
 '%debug',
 '%history',
 '%lprun',
 '%lsmagic',
 '%magic',
 '%matplotlib',
 '%memit',
 '%mode',
 '%mprun',
 '%paste',
 '%prun',
 '%run',
 '%time',
 '%timeit',
 '& (ampersand)',
 '* (asterisk)',
 ': (colon)',
 '? (question mark)',
 '?? (double question mark)',
 '_ (underscore) shortcut',
 '| (operator)',
 'A absolute value function',
 'aggregate() method',
 'aggregates computed directly from object',
 'multidimensional',
 'summarizing set of values with',
 'aggregation (NumPy)',
 'minimum and maximum',
 'multidimensional aggregates',
 'presidents average height example',
 'summing the values in an array',
 'various functions',
 'aggregation (Pandas)',
 'groupby() operation',
 'MultiIndex',
 'Planets dataset for',
 'simple aggregation',
 'Akaike information criterion (AIC)',
 'Albers equal-area projection',
 'algorithmic efficiency big-O notation',
 'dataset size and',
 'ampersand (&)',
 'Anaconda',
 'xiv and keyword',
 'annotation of plots',
 'arrows',

In [6]:
concepts_found

{'Bayesian',
 'Bell',
 'Bias',
 'Boolean',
 'Calc',
 'Central Limit Theorem',
 'Clustering',
 'Compute',
 'Eigenvalue',
 'Eigenvector',
 'Engage',
 'Excel',
 'Facebook',
 'Google',
 'Internet',
 'Learning',
 'Matplotlib',
 'Median',
 'Microsoft',
 'Netflix',
 'Networks',
 'Note',
 'PCA',
 'Pandas',
 'Python',
 'State',
 'Storage',
 'Trace',
 'Twitter',
 'US',
 'Variance',
 'Word',
 'absolute value',
 'access',
 'analysis',
 'application development',
 'applications',
 'attributes',
 'authentication',
 'authorization',
 'billing',
 'boosting',
 'categories of',
 'challenges',
 'characteristics',
 'classification',
 'classification task',
 'clients',
 'clustering',
 'collaboration',
 'communications',
 'compliance',
 'components',
 'computing',
 'confusion matrix',
 'connectivity',
 'controlling',
 'creating',
 'cross-validation',
 'data analysis',
 'data as',
 'data integrity',
 'data management',
 'data point',
 'data science',
 'data security',
 'data services',
 'datasets',
 'debuggi